***This Gridsearch drop migration rate***

In [102]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression
import seaborn as sns
import joblib
from termcolor import colored
import mlflow
from memoized_property import memoized_property
from mlflow.tracking import MlflowClient
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.svm import SVR

In [103]:
df = gpd.read_file('../YouthInTheCity/data/regression_gdf.shp')

# Select features manually

df['extra_edu']=df.loc[:,'education']

df.drop(columns=['schools', 'universiti', 'kindergart','HK_EU15', 'HK_EU28', 'HK_Polen', 'HK_EheJug', 'HK_EheSU', 'HK_Turk',
              'schools', 'universiti','kindergart','night_life',
                       'mig_rate','HK_Arab', 'HK_Sonst', 'HK_NZOrd', 'BZR_NAME', 
              'noise',  'health_car', 'vegpm10', 'change','ch_rate',
            'public_tra', 'green' ,'other_sch', 'secon_sch','vocat_sch',
                'sp_nee_sch', 'community','night_life', 'water',  'ave_rent',
              'public_ser','dyn_ew', 'dyn_unempl', 'priv_schoo', 'primary_sc',
                        'five_y_pls', 'dyn_sales','air', 'education','eating','bio'], inplace=True)
df

,PLR_ID,culture,outdoor_fa,outdoor_le,social_hou,public_hou,child_pov,vegpm20,Gymnasium,kita,stations,geometry,extra_edu
0,11501341,2.0,224.0,8.0,1.302637,24.830919,2.58,3.456812,0.0,20.0,10.0,"POLYGON ((399848.954 5815619.506, 399854.727 5...",1.0
1,3701659,12.0,130.0,10.0,0.390507,12.771489,8.13,2.896211,1.0,47.0,13.0,"POLYGON ((393175.012 5821135.577, 393167.174 5...",6.0
2,7601340,1.0,4.0,6.0,12.720000,26.730000,41.68,2.648765,0.0,17.0,0.0,"POLYGON ((388394.110 5807792.070, 388379.276 5...",0.0
3,2500831,7.0,221.0,14.0,0.740000,8.370000,21.15,2.706697,2.0,41.0,13.0,"POLYGON ((396010.552 5819339.570, 396021.567 5...",2.0
4,12601134,0.0,85.0,24.0,1.560000,42.230000,51.53,3.861459,1.0,24.0,2.0,"POLYGON ((387367.428 5828619.763, 387315.862 5...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,12500927,0.0,61.0,9.0,1.560000,42.230000,20.69,5.223326,2.0,27.0,4.0,"POLYGON ((387280.650 5828614.114, 387266.882 5...",0.0
532,3200204,0.0,75.0,8.0,3.209599,3.458242,4.24,3.389884,0.0,7.0,2.0,"POLYGON ((392586.179 5829994.574, 392586.626 5...",0.0
533,10400941,1.0,45.0,4.0,0.550044,0.751582,2.89,2.551829,0.0,12.0,12.0,"POLYGON ((406064.851 5815581.891, 406061.455 5...",0.0
534,8100101,18.0,342.0,1.0,6.729473,9.548275,40.42,6.286957,1.0,47.0,3.0,"POLYGON ((391887.140 5816592.598, 391898.686 5...",2.0


In [8]:
df.set_index('PLR_ID')

,culture,outdoor_fa,outdoor_le,social_hou,public_hou,child_pov,vegpm20,Gymnasium,kita,stations,geometry,extra_edu
PLR_ID,,,,,,,,,,,,
11501341,2.0,224.0,8.0,1.302637,24.830919,2.58,3.456812,0.0,20.0,10.0,"POLYGON ((399848.954 5815619.506, 399854.727 5...",1.0
3701659,12.0,130.0,10.0,0.390507,12.771489,8.13,2.896211,1.0,47.0,13.0,"POLYGON ((393175.012 5821135.577, 393167.174 5...",6.0
7601340,1.0,4.0,6.0,12.720000,26.730000,41.68,2.648765,0.0,17.0,0.0,"POLYGON ((388394.110 5807792.070, 388379.276 5...",0.0
2500831,7.0,221.0,14.0,0.740000,8.370000,21.15,2.706697,2.0,41.0,13.0,"POLYGON ((396010.552 5819339.570, 396021.567 5...",2.0
12601134,0.0,85.0,24.0,1.560000,42.230000,51.53,3.861459,1.0,24.0,2.0,"POLYGON ((387367.428 5828619.763, 387315.862 5...",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
12500927,0.0,61.0,9.0,1.560000,42.230000,20.69,5.223326,2.0,27.0,4.0,"POLYGON ((387280.650 5828614.114, 387266.882 5...",0.0
3200204,0.0,75.0,8.0,3.209599,3.458242,4.24,3.389884,0.0,7.0,2.0,"POLYGON ((392586.179 5829994.574, 392586.626 5...",0.0
10400941,1.0,45.0,4.0,0.550044,0.751582,2.89,2.551829,0.0,12.0,12.0,"POLYGON ((406064.851 5815581.891, 406061.455 5...",0.0


In [19]:
y = df["child_pov"]
X = df.drop(columns=["child_pov", 'geometry'])

## Gridsearch with KNN regressor

In [20]:
pipe = Pipeline([
    ('scaler', RobustScaler()),
    ('model', KNeighborsRegressor())
])

In [13]:
#use get_paraam to find the specific name of the step for grid/pipe
pipe.get_params()

{'memory': None,
 'steps': [('scaler', RobustScaler()), ('model', KNeighborsRegressor())],
 'verbose': False,
 'scaler': RobustScaler(),
 'model': KNeighborsRegressor(),
 'scaler__copy': True,
 'scaler__quantile_range': (25.0, 75.0),
 'scaler__unit_variance': False,
 'scaler__with_centering': True,
 'scaler__with_scaling': True,
 'model__algorithm': 'auto',
 'model__leaf_size': 30,
 'model__metric': 'minkowski',
 'model__metric_params': None,
 'model__n_jobs': None,
 'model__n_neighbors': 5,
 'model__p': 2,
 'model__weights': 'uniform'}

In [36]:
# Grid search KNNImputer parameter n_neighbors
grid_search = GridSearchCV(
    pipe,
    param_grid={
        'model__algorithm': ['auto', 'ball_tree', 'kd_tree'],
        'model__weights': ['uniform', 'distance'],
        'model__n_neighbors': np.arange(5,50),
        'model__n_jobs': [-1]})

In [38]:
grid_search.fit(X, y)

# Cross validate optimal pipeline
cv_score_knn = cross_validate(grid_search.best_estimator_,
                X, y,cv=5,
                scoring= ['r2','neg_mean_absolute_error', 'neg_mean_squared_error'])


{'model__algorithm': 'auto',
 'model__n_jobs': -1,
 'model__n_neighbors': 9,
 'model__weights': 'distance'}

In [56]:
cv_score_knn['test_neg_mean_squared_error'].mean()

-177.8932981337642

In [61]:
cv_score_knn['test_neg_mean_absolute_error'].mean()

-10.476671533963806

In [60]:
cv_score_knn['test_r2'].mean()

0.3493669155967033

In [55]:
cv_score_knn[]

{'fit_time': array([0.00720096, 0.00757813, 0.0075438 , 0.01045418, 0.00980091]),
 'score_time': array([0.00886297, 0.01044583, 0.00939608, 0.01530695, 0.0185411 ]),
 'test_r2': array([0.29428915, 0.35638074, 0.25681899, 0.38463947, 0.45470623]),
 'test_neg_mean_absolute_error': array([-11.36330033, -10.51905412,  -9.78512759, -10.23246581,
        -10.48340982]),
 'test_neg_mean_squared_error': array([-209.42870187, -177.59950595, -159.5477041 , -173.45893174,
        -169.43164701])}

In [23]:
cv_score_knn_r = cross_val_score(grid_search.best_estimator_,
                X, y,cv=5,
                scoring= 'r2').mean()
cv_score_knn_r

0.3493669155967033

In [24]:
cv_score_knn_mse = cross_val_score(grid_search.best_estimator_,
                X, y,cv=5,
                scoring= 'neg_mean_squared_error').mean()
cv_score_knn_mse

-177.8932981337642

In [25]:
cv_score_knn

-10.476671533963806

# Gridsearch with SVM. SVR

## Gridsearch with SVC

In [62]:
from sklearn.svm import SVR

pipe_svr = Pipeline([
    ('scaler', RobustScaler()),
    ('model', SVR())
])

In [81]:
pipe_svr.get_params()

{'memory': None,
 'steps': [('scaler', RobustScaler()), ('model', SVR())],
 'verbose': False,
 'scaler': RobustScaler(),
 'model': SVR(),
 'scaler__copy': True,
 'scaler__quantile_range': (25.0, 75.0),
 'scaler__unit_variance': False,
 'scaler__with_centering': True,
 'scaler__with_scaling': True,
 'model__C': 1.0,
 'model__cache_size': 200,
 'model__coef0': 0.0,
 'model__degree': 3,
 'model__epsilon': 0.1,
 'model__gamma': 'scale',
 'model__kernel': 'rbf',
 'model__max_iter': -1,
 'model__shrinking': True,
 'model__tol': 0.001,
 'model__verbose': False}

In [82]:
# Grid search KNNImputer parameter n_neighbors
grid_search_svr = GridSearchCV(
    pipe_svr,
    param_grid={
        'model__C': [0.01, 0.1, 1.0, 1.2, 1,5],
        'model__kernel': ['rbf', 'linear', 'poly','sigmoid'],
        'model__degree': [2,3,4,5,6,7],
        'model__gamma': ['scale', 'auto'],
        })
grid_search_svr.fit(X, y)

# Cross validate optimal pipeline
cv_score_svr = cross_validate(grid_search_svr.best_estimator_,
                X, y,cv=5,
                scoring= ['r2','neg_mean_absolute_error', 'neg_mean_squared_error'])


In [68]:
cv_score_svr['test_neg_mean_squared_error'].mean()

-189.03341631302447

In [69]:
cv_score_svr['test_neg_mean_absolute_error'].mean()

-10.628194898085844

In [70]:
cv_score_svr['test_r2'].mean()

0.30933240083208435

In [67]:
grid_search_svr.best_params_  #--> cv = -10.35

{'model__C': 5,
 'model__degree': 2,
 'model__gamma': 'scale',
 'model__kernel': 'rbf'}

In [85]:
pipe_sgd = Pipeline([
    ('scaler', RobustScaler()),
    ('model', SGDRegressor())
])

In [90]:
# Grid search KNNImputer parameter n_neighbors
grid_search_sgd = GridSearchCV(
    pipe_sgd,
    param_grid={
        'model__penalty': ['l2', 'l1','elasticnet'],
        'model__alpha': [0.001, 0.0001, 0.00001],
        'model__learning_rate': ['invscaling','constant','optimal','adaptive'],
        })
grid_search_sgd.fit(X, y)

GridSearchCV(estimator=Pipeline(steps=[('scaler', RobustScaler()),
                                       ('model', SGDRegressor())]),
             param_grid={'model__alpha': [0.001, 0.0001, 1e-05],
                         'model__learning_rate': ['invscaling', 'constant',
                                                  'optimal', 'adaptive'],
                         'model__penalty': ['l2', 'l1', 'elasticnet']})

In [98]:
cv_score_sgd = cross_validate(grid_search_sgd.best_estimator_,
                X, y,cv=5,
                scoring= ['r2','neg_mean_absolute_error', 'neg_mean_squared_error'])

In [99]:
cv_score_sgd['test_neg_mean_squared_error'].mean()

-187.64662289094127

In [100]:
cv_score_sgd['test_neg_mean_absolute_error'].mean()

-10.799247139312161

In [101]:
cv_score_sgd['test_r2'].mean()

0.3151909317590976

In [91]:
grid_search_sgd.best_params_

{'model__alpha': 0.001,
 'model__learning_rate': 'invscaling',
 'model__penalty': 'elasticnet'}

## Gridsearch with Linear model

In [20]:
pipe_linear = Pipeline([
    ('scaler', RobustScaler()),
    ('model', LinearRegression())
])

In [122]:
pipe_linear.get_params()

{'memory': None,
 'steps': [('scaler', RobustScaler()), ('model', LinearRegression())],
 'verbose': False,
 'scaler': RobustScaler(),
 'model': LinearRegression(),
 'scaler__copy': True,
 'scaler__quantile_range': (25.0, 75.0),
 'scaler__unit_variance': False,
 'scaler__with_centering': True,
 'scaler__with_scaling': True,
 'model__copy_X': True,
 'model__fit_intercept': True,
 'model__n_jobs': None,
 'model__normalize': 'deprecated',
 'model__positive': False}

In [21]:
# Grid search KNNImputer parameter n_neighbors
grid_search_linear = GridSearchCV(
    pipe_linear,
    param_grid={
        'model__fit_intercept': [True, False],
        'model__n_jobs':[ -1]})
grid_search_linear.fit(X, y)

# Cross validate optimal pipeline
cv_score_linear = cross_val_score(grid_search_linear.best_estimator_,
                X, y,cv=5,
                scoring= 'neg_mean_absolute_error').mean()
cv_score_linear

-10.91446801856572

In [22]:
cv_score_linear_r = cross_val_score(grid_search_linear.best_estimator_,
                X, y,cv=5,
                scoring= 'r2').mean()
cv_score_linear_r

0.2996728693837336

In [25]:
cv_score_linear_mse = cross_val_score(grid_search_linear.best_estimator_,
                X, y,cv=5,
                scoring= 'neg_mean_squared_error').mean()
cv_score_linear_mse

-191.2981368419528

In [26]:
grid_search_linear.best_params_

{'model__fit_intercept': True, 'model__n_jobs': -1}

In [60]:
neg_mean = {"Linear model":cv_score_linear, 
            "KNNRegressor": cv_score_knn,
           "SGDRegressor":cv_score_sgd,
           "SVR": cv_score_svr,
           "SVR_linear": cv_score_svr2}
neg_mean_error = pd.DataFrame(data = neg_mean, index=[0]).T
neg_mean_error.rename(columns={0:"neg_mean_absolute_error"}, inplace=True)

In [61]:
neg_mean_error["r2_score"] = [cv_score_linear_r, cv_score_knn_r, cv_score_sgd_r, cv_score_svr_r, cv_score_svr_l_r]
neg_mean_error

,neg_mean_absolute_error,r2_score
Linear model,-10.914468,0.299673
KNNRegressor,-9.628495,0.404332
SGDRegressor,-11.048812,0.267577
SVR,-10.354776,0.338345
SVR_linear,-10.992459,0.266438


In [62]:
neg_mean_error["mse"] = [cv_score_linear_mse, cv_score_knn_mse, cv_score_sgd_mse, cv_score_svr_mse, cv_score_svr_l_mse]
neg_mean_error

,neg_mean_absolute_error,r2_score,mse
Linear model,-10.914468,0.299673,-191.298137
KNNRegressor,-9.628495,0.404332,-160.730417
SGDRegressor,-11.048812,0.267577,-197.278558
SVR,-10.354776,0.338345,-179.648908
SVR_linear,-10.992459,0.266438,-200.671826


In [86]:
models = [LinearRegression(), KNeighborsRegressor(), SGDRegressor()]
metrics_df = pd.DataFrame(columns = ["mean_abs_error", "mean_squ_error", "r_score"])
for model in models:
    preproc_pipe = Pipeline([
            ('knnimputer', KNNImputer(missing_values= np.nan)),
            ('robustscaler', RobustScaler())
        ])
    pipeline = Pipeline([
            ('preproc', preproc_pipe),
            ('model', model)
        ])
    y = df["child_pov"]
    X = df.drop(columns=["child_pov"])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    r_score = r2_score(y_test, y_pred)
    mean_abs_error = mean_absolute_error(y_test, y_pred)
    mean_squ_error = mean_squared_error(y_test, y_pred)
    metrics_df = pd.DataFrame(data = [mean_abs_error, mean_squ_error, r_score], axis=1)
metrics_df

TypeError: __init__() got an unexpected keyword argument 'axis'

In [85]:
metrics_dft = pd.DataFrame(columns = ["mean_abs_error", "mean_squ_error", "r_score"])
metrics_dft

,mean_abs_error,mean_squ_error,r_score


In [13]:
EXPERIMENT_NAME = "non-spatial_regression"
yourname = "Batch_874_Batch_874_Youth_in_the_city"
MLFLOW_URI = "https://mlflow.lewagon.ai/"
class Trainer(object):
    def __init__(self, X, y):
        """
            X: pandas DataFrame --> drop the "geometry and the "Kinderarmut"
            y: pandas Series --> "Kinderarmut"
        """
        self.pipeline = None
        self.X = X
        self.y = y
        # for MLFlow
        self.experiment_name = EXPERIMENT_NAME

    def set_experiment_name(self, experiment_name):
        '''defines the experiment name for MLFlow'''
        self.experiment_name = experiment_name

    def set_pipeline(self):
        """defines the pipeline as a class attribute"""
        preproc_pipe = Pipeline([
            ('knnimputer', KNNImputer(missing_values= np.nan)),
            ('robustscaler', RobustScaler())
        ])
        self.pipeline = Pipeline([
            ('preproc', preproc_pipe),
            ('linear_model', LinearRegression())
        ])

    def run(self):
        self.set_pipeline()
        self.mlflow_log_param("model", "Linear")
        self.pipeline.fit(self.X, self.y)

    def evaluate(self, X_test, y_test):
        """evaluates the pipeline on df_test and return the r2 score"""
        y_pred = self.pipeline.predict(X_test)
        r_score = r2_score(y_test, y_pred)
        mean_abs_error = mean_absolute_error(y_test, y_pred)
        mean_squ_error = mean_squared_error(y_test, y_pred)
        self.mlflow_log_metric("mean_abs_error", mean_abs_error)
        metrics_df = pd.DataFrame(data = [mean_abs_error, mean_squ_error, r_score])
        return metrics_df

    def save_model_locally(self):
        """Save the model into a .joblib format"""
        joblib.dump(self.pipeline, 'model.joblib')
        print(colored("model.joblib saved locally", "green"))

    # MLFlow methods
    @memoized_property
    def mlflow_client(self):
        mlflow.set_tracking_uri(MLFLOW_URI)
        return MlflowClient()

    @memoized_property
    def mlflow_experiment_id(self):
        try:
            return self.mlflow_client.create_experiment(self.experiment_name)
        except BaseException:
            return self.mlflow_client.get_experiment_by_name(
                self.experiment_name).experiment_id

    @memoized_property
    def mlflow_run(self):
        return self.mlflow_client.create_run(self.mlflow_experiment_id)

    def mlflow_log_param(self, key, value):
        self.mlflow_client.log_param(self.mlflow_run.info.run_id, key, value)

    def mlflow_log_metric(self, key, value):
        self.mlflow_client.log_metric(self.mlflow_run.info.run_id, key, value)

In [54]:
y = df["child_pov"]
X = df.drop(columns=["child_pov"])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# Train and save model, locally and
trainer = Trainer(X=X_train, y=y_train)
trainer.set_experiment_name('linear')
trainer.run()
linear = trainer.evaluate(X_test, y_test)
print(linear)
trainer.save_model_locally()

           0
0   7.128310
1  82.027018
2   0.684207
model.joblib saved locally


In [15]:
EXPERIMENT_NAME = "non-spatial_regression"
yourname = "Batch_874_Batch_874_Youth_in_the_city"
MLFLOW_URI = "https://mlflow.lewagon.ai/"
class Trainer_KNN(object):
    def __init__(self, X, y):
        """
            X: pandas DataFrame --> drop the "geometry and the "Kinderarmut"
            y: pandas Series --> "Kinderarmut"
        """
        self.pipeline = None
        self.X = X
        self.y = y
        # for MLFlow
        self.experiment_name = EXPERIMENT_NAME

    def set_experiment_name(self, experiment_name):
        '''defines the experiment name for MLFlow'''
        self.experiment_name = experiment_name

    def set_pipeline(self):
        """defines the pipeline as a class attribute"""
        preproc_pipe = Pipeline([
            ('knnimputer', KNNImputer(missing_values= np.nan)),
            ('robustscaler', RobustScaler())
        ])
        self.pipeline = Pipeline([
            ('preproc', preproc_pipe),
            ('KNN_model', KNeighborsRegressor())
        ])

    def run(self):
        self.set_pipeline()
        self.mlflow_log_param("model", "Linear")
        self.pipeline.fit(self.X, self.y)

    def evaluate(self, X_test, y_test):
        """evaluates the pipeline on df_test and return the r2 score"""
        y_pred = self.pipeline.predict(X_test)
        r_score = r2_score(y_test, y_pred)
        mean_abs_error = mean_absolute_error(y_test, y_pred)
        mean_squ_error = mean_squared_error(y_test, y_pred)
        self.mlflow_log_metric("mean_abs_error", mean_abs_error)
        metrics_df = pd.DataFrame(data = [mean_abs_error, mean_squ_error, r_score])
        return metrics_df

    def save_model_locally(self):
        """Save the model into a .joblib format"""
        joblib.dump(self.pipeline, 'model.joblib')
        print(colored("model.joblib saved locally", "green"))

    # MLFlow methods
    @memoized_property
    def mlflow_client(self):
        mlflow.set_tracking_uri(MLFLOW_URI)
        return MlflowClient()

    @memoized_property
    def mlflow_experiment_id(self):
        try:
            return self.mlflow_client.create_experiment(self.experiment_name)
        except BaseException:
            return self.mlflow_client.get_experiment_by_name(
                self.experiment_name).experiment_id

    @memoized_property
    def mlflow_run(self):
        return self.mlflow_client.create_run(self.mlflow_experiment_id)

    def mlflow_log_param(self, key, value):
        self.mlflow_client.log_param(self.mlflow_run.info.run_id, key, value)

    def mlflow_log_metric(self, key, value):
        self.mlflow_client.log_metric(self.mlflow_run.info.run_id, key, value)

In [16]:
#KNN
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# Train and save model, locally and
trainer_knn = Trainer(X=X_train, y=y_train)
trainer_knn.set_experiment_name('KNN')
trainer_knn.run()
knn_reg = trainer_knn.evaluate(X_test, y_test)
print(knn_reg)
trainer_knn.save_model_locally()

           0
0   7.853060
1  95.744934
2   0.646386
model.joblib saved locally


In [17]:
EXPERIMENT_NAME = "non-spatial_regression"
yourname = "Batch_874_Batch_874_Youth_in_the_city"
MLFLOW_URI = "https://mlflow.lewagon.ai/"
class Trainer_SGD(object):
    def __init__(self, X, y):
        """
            X: pandas DataFrame --> drop the "geometry and the "Kinderarmut"
            y: pandas Series --> "Kinderarmut"
        """
        self.pipeline = None
        self.X = X
        self.y = y
        # for MLFlow
        self.experiment_name = EXPERIMENT_NAME

    def set_experiment_name(self, experiment_name):
        '''defines the experiment name for MLFlow'''
        self.experiment_name = experiment_name

    def set_pipeline(self):
        """defines the pipeline as a class attribute"""
        preproc_pipe = Pipeline([
            ('knnimputer', KNNImputer(missing_values= np.nan)),
            ('robustscaler', RobustScaler())
        ])
        self.pipeline = Pipeline([
            ('preproc', preproc_pipe),
            ('KNN_model', SGDRegressor())
        ])

    def run(self):
        self.set_pipeline()
        self.mlflow_log_param("model", "SGD")
        self.pipeline.fit(self.X, self.y)

    def evaluate(self, X_test, y_test):
        """evaluates the pipeline on df_test and return the r2 score"""
        y_pred = self.pipeline.predict(X_test)
        r_score = r2_score(y_test, y_pred)
        mean_abs_error = mean_absolute_error(y_test, y_pred)
        mean_squ_error = mean_squared_error(y_test, y_pred)
        self.mlflow_log_metric("mean_abs_error", mean_abs_error)
        metrics_df = pd.DataFrame(data = [mean_abs_error, mean_squ_error, r_score])
        return metrics_df

    def save_model_locally(self):
        """Save the model into a .joblib format"""
        joblib.dump(self.pipeline, 'model_sgd.joblib')
        print(colored("model_sgd.joblib saved locally", "green"))

    # MLFlow methods
    @memoized_property
    def mlflow_client(self):
        mlflow.set_tracking_uri(MLFLOW_URI)
        return MlflowClient()

    @memoized_property
    def mlflow_experiment_id(self):
        try:
            return self.mlflow_client.create_experiment(self.experiment_name)
        except BaseException:
            return self.mlflow_client.get_experiment_by_name(
                self.experiment_name).experiment_id

    @memoized_property
    def mlflow_run(self):
        return self.mlflow_client.create_run(self.mlflow_experiment_id)

    def mlflow_log_param(self, key, value):
        self.mlflow_client.log_param(self.mlflow_run.info.run_id, key, value)

    def mlflow_log_metric(self, key, value):
        self.mlflow_client.log_metric(self.mlflow_run.info.run_id, key, value)

In [18]:
#SGD
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
# Train and save model, locally and
trainer_sgd = Trainer(X=X_train, y=y_train)
trainer_sgd.set_experiment_name('SGD')
trainer_sgd.run()
sgd_reg = trainer_knn.evaluate(X_test, y_test)
print(sgd_reg)
trainer_sgd.save_model_locally()

           0
0   7.947480
1  91.630146
2   0.681859
model.joblib saved locally


In [73]:
trainer_knn.summary

AttributeError: 'Trainer' object has no attribute 'summary'

In [38]:
metrics_df = linear.merge(knn_reg)
metrics_df

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [51]:
knn_df = pd.DataFrame(knn_reg)
sgd_df = pd.DataFrame(sgd_reg)
linear_df = pd.DataFrame(linear)

In [55]:
linear

,0
0,7.128310
1,82.027018
2,0.684207


In [67]:
metrics_df = pd.concat([linear, knn_df,sgd_df],axis = 1)
metrics_df#.rename(columns = {0:"linear", 0: "KNN1", 2: "SGD" }, inplace=True)

,0,0,0
0,7.128310,7.853060,7.947480
1,82.027018,95.744934,91.630146
2,0.684207,0.646386,0.681859


In [70]:
#metrics_df.rename(columns = ["linear", "KNN", "SGD"], inplace=True)
metrics_df.rename(index={0: "mean_abs_error", 1: "mean_squ_error", 2: "r2_score" }, inplace=True)
metrics_df

,0,0,0
mean_abs_error,7.128310,7.853060,7.947480
mean_squ_error,82.027018,95.744934,91.630146
r2_score,0.684207,0.646386,0.681859


In [72]:
metrics_df.rename(columns = {0:"linear", 0:"KNN", 0:"SGD"}, inplace=True)
metrics_df

,2,2,2
mean_abs_error,7.128310,7.853060,7.947480
mean_squ_error,82.027018,95.744934,91.630146
r2_score,0.684207,0.646386,0.681859


In [ ]:
#metrics_df = metrics_df.T
#metrics_df.rename(columns = {0:"mean_abs_error", 1: "mean_squ_error", 2: "r2_score" }, inplace=True)
#metrics_df.set_index("linear", inplace=True)
metrics_df.rename(columns = {"mean_abs_error":linear}, inplace=True)

In [30]:
metrics_df.append(knn_reg.T)

/var/folders/n9/s2d67hcd6m30qbm0pf_jfqy00000gn/T/ipykernel_54592/710091197.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df.append(knn_reg.T)


,mean_abs_error,mean_squ_error,r2_score,0,1,2
0,7.430035,86.164902,0.681061,NaN,NaN,NaN
0,NaN,NaN,NaN,7.85306,95.744934,0.646386
